# 1. Import libraries

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import data
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
import os

# 2. Importing the dataset
### You need to change Ion and data path to simulate ion constituents. Ion = '##' & df = pd.read_csv(##_path)

In [2]:
ion_types = ["Alkalinity", "Br", "Ca", "Cl", "K", "Mg", "Na", "SO4" "TDS"]
paths = [f"inputs/{ion_type}.csv" for ion_type in ion_types]

Ion = 'Alkalinity'

df = pd.read_csv(f'inputs/{Ion}.csv')
df = pd.DataFrame(df)
df["Collection Date"] = pd.to_datetime(df["Collection Date"])
df["Long Station Name"] = df["Long Station Name"].astype("category")
df["Location"] = df["Location"].astype("category")
df["Area"] = df["Area"].astype("category")
df["month"] = df["Collection Date"].dt.month_name()
df["month"] = df["month"].astype("category")
df["year"] = df["Collection Date"].dt.year
df["year"] = df["year"].astype("category")
df["year"] = df["year"].astype("category")
df["WYT"] = df["WYT"].astype("category")
df["EC"] = df["EC"].astype("float64")
df[Ion] = df[Ion].astype("float64")


df

,Long Station Name,Collection Date,EC,Alkalinity,Sacramento X2,Location,Area,WYT,Data,month,year
0,Old River near Rock Slough,1959-06-19,318.0,76.0,91.438186,OMR,Interior,BN,TT,June,1959
1,Old River near Rock Slough,1959-06-19,327.0,73.0,91.438186,OMR,Interior,BN,TT,June,1959
2,Old River near Rock Slough,1959-07-07,716.0,74.0,94.298896,OMR,Interior,BN,TT,July,1959
3,Old River near Rock Slough,1959-07-07,749.0,73.0,94.298896,OMR,Interior,BN,TT,July,1959
4,Old River near Rock Slough,1959-07-21,1320.0,69.0,95.924951,OMR,Interior,BN,TT,July,1959
...,...,...,...,...,...,...,...,...,...,...,...
1034,PDUP,2020-04-29,908.0,110.0,75.000000,SouthDelta,Interior,D,NCRO,April,2020
1035,TWA,2020-04-29,676.0,98.0,75.000000,SouthDelta,Interior,D,NCRO,April,2020
1036,PDUP,2020-05-18,1540.0,166.0,81.000000,SouthDelta,Interior,D,NCRO,May,2020
1037,ORX,2020-06-16,254.0,50.0,80.000000,SouthDelta,Interior,D,NCRO,June,2020


# 3. Splitting the dataset into the Training set and Test set
### &
## Feature Scaling

### In this study, Training and Test datasets were randomly selected (20% Test & 80% Training)

In [4]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
ohe = OneHotEncoder()
feature_array = ohe.fit_transform(df[['WYT','Location','month']]).toarray()
feature_labels = ohe.categories_
feature_labels = np.concatenate((feature_labels[0], feature_labels[1],feature_labels[2]))
XD = pd.DataFrame(feature_array,columns=feature_labels)
dfmerged=pd.merge(df,XD, left_index=True, right_index=True)
dfmerged.dropna(subset=["Sacramento X2"], inplace=True)
# dfmerged=dfmerged.loc[:, ['EC', 'Sacramento X2',Ion, 'AN', 'BN', 'C', 'D', 'W', 'OMR','SJRcorridor', 'SouthDelta', 'April', 'August', 'December', 'February','January', 'July', 'June', 'March',
#  'May', 'November', 'October','September']]
X = dfmerged.loc[:, ['EC', 'Sacramento X2', 'AN', 'BN', 'C', 'D', 'W', 'OMR','SJRcorridor', 'SouthDelta', 'April', 'August', 'December', 'February','January', 'July', 'June', 'March','May', 'November', 'October','September']]
y=dfmerged.iloc[:, 3]

X2=X

X2['EC']=(X2['EC'] -50)/(3500-50)
X2['Sacramento X2']=(X2['Sacramento X2'] -0)/(100-0)


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X2, y, test_size=0.2, random_state=1)


print('df.shape=',df.shape)
print('dfmerged.shape=',dfmerged.shape)
print('X.shape=',X.shape)
print('XD.shape=',XD.shape)
print('X_Train.shape=',X_train.shape)
print('y_Train.shape=',y_train.shape)
print('X_Test.shape=',X_test.shape)
print('y_Test.shape=',y_test.shape)
dfmerged

df.shape= (1039, 11)
dfmerged.shape= (1039, 31)
X.shape= (1039, 22)
XD.shape= (1039, 20)
X_Train.shape= (831, 22)
y_Train.shape= (831,)
X_Test.shape= (208, 22)
y_Test.shape= (208,)


,Long Station Name,Collection Date,EC,Alkalinity,Sacramento X2,Location,Area,WYT,Data,month,...,December,February,January,July,June,March,May,November,October,September
0,Old River near Rock Slough,1959-06-19,318.0,76.0,91.438186,OMR,Interior,BN,TT,June,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Old River near Rock Slough,1959-06-19,327.0,73.0,91.438186,OMR,Interior,BN,TT,June,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Old River near Rock Slough,1959-07-07,716.0,74.0,94.298896,OMR,Interior,BN,TT,July,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Old River near Rock Slough,1959-07-07,749.0,73.0,94.298896,OMR,Interior,BN,TT,July,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Old River near Rock Slough,1959-07-21,1320.0,69.0,95.924951,OMR,Interior,BN,TT,July,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,PDUP,2020-04-29,908.0,110.0,75.000000,SouthDelta,Interior,D,NCRO,April,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1035,TWA,2020-04-29,676.0,98.0,75.000000,SouthDelta,Interior,D,NCRO,April,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1036,PDUP,2020-05-18,1540.0,166.0,81.000000,SouthDelta,Interior,D,NCRO,May,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1037,ORX,2020-06-16,254.0,50.0,80.000000,SouthDelta,Interior,D,NCRO,June,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


# 4. Alternative Models
### 1- Multiple Linear Regression (MLR), 2- Regression Trees (RT), 3- Random Forest (RF), 4- Gradient Boosting (GB),  5- Artificial Neural Networks (ANN)

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

regressorLinear = LinearRegression()
regressorTree = DecisionTreeRegressor(random_state=0, min_samples_leaf=4)
regressorRF = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, random_state=0)
regressorGB = GradientBoostingRegressor(min_samples_leaf=4)


print(Ion)

regressorLinear.fit(X_train, y_train)
y_pred_test = regressorLinear.predict(X_test)
y_pred_train = regressorLinear.predict(X_train)
y_pred_LR = regressorLinear.predict(X)
y_pred_test_LR = regressorLinear.predict(X_test)
# print("LinearRegression:")
# print("        ", "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
# print("        ", "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_test, y_pred_test), ndigits=2), "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))


# print("        ")

regressorTree.fit(X_train, y_train)
y_pred_test = regressorTree.predict(X_test)
y_pred_train = regressorTree.predict(X_train)
y_pred_RT = regressorTree.predict(X)
y_pred_test_RT = regressorTree.predict(X_test)
# print("DecisionTreeRegressor:")
# print("        ", "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
# print("        ", "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_test, y_pred_test), ndigits=2), "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))

# print("        ")

regressorRF.fit(X_train, y_train)
y_pred_test = regressorRF.predict(X_test)
y_pred_train = regressorRF.predict(X_train)
y_pred_RF = regressorRF.predict(X)
y_pred_test_RF = regressorRF.predict(X_test)
# print("Random Forest:")
# print("        ", "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
# print("        ", "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_test, y_pred_test), ndigits=2), "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))

print("        ")

regressorGB.fit(X_train, y_train)
y_pred_test = regressorGB.predict(X_test)
y_pred_train = regressorGB.predict(X_train)
y_pred_GB = regressorGB.predict(X)
y_pred_test_GB = regressorGB.predict(X_test)
# print("GradientBoosting:")
# print("        ", "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
# print("        ", "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2), "  MAE =", round(
#     mean_absolute_error(y_test, y_pred_test), ndigits=2), "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))

# print("        ")


Alkalinity
        


# ANN

In [13]:
if Ion=="TDS":
    N=[30,30,20,30]
    A=['elu','sigmoid','elu','relu']
elif Ion == "Mg":
    N=[30,30,20,30]
    A=['relu','elu','tanh','relu']
elif Ion == "Na":
    N=[30,30,20,30]
    A=['tanh','elu','sigmoid','elu']
elif Ion == "Ca":
    N=[40,40,40,30]
    A=['elu','sigmoid','relu','tanh']
elif Ion == "Cl":
    N=[30,20,30,30]
    A=['relu','elu','sigmoid','elu']
elif Ion == "SO4":
    N=[44,44,44,22]
    A=['relu','relu','relu','relu']
elif Ion == "Br":
    N=[20,20,20,20]
    A=['elu','sigmoid','elu','tanh']
elif Ion == "Alkalinity":
    N=[30,30,30,30]
    A=['tanh','relu','tanh','elu']
elif Ion == "K":
    N=[44,44,44,22]
    A=['relu','relu','relu','relu']



In [14]:
root_logdir = os.path.join(os.curdir, "tf_training_logs")
tensorboard_cb = keras.callbacks.TensorBoard(root_logdir)

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=22))
ann.add(tf.keras.layers.Dense(units=N[0], activation=A[0]))
ann.add(tf.keras.layers.Dense(units=N[1], activation=A[1]))
ann.add(tf.keras.layers.Dense(units=N[2], activation=A[2]))
ann.add(tf.keras.layers.Dense(units=N[3], activation=A[3]))
ann.add(tf.keras.layers.Dense(units=1))
ann.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.001), loss="mean_absolute_error", metrics=['mean_absolute_error'])
history = ann.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[keras.callbacks.EarlyStopping( monitor="val_loss", patience=100, mode="min", restore_best_weights=True),tensorboard_cb],batch_size=32, epochs=5000, verbose=0)


y_pred_test = ann.predict(X_test)
y_pred_train = ann.predict(X_train)

print(Ion, "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(mean_absolute_error(
    y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
print(Ion, "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2),  "  MAE =", round(mean_absolute_error(
    y_test, y_pred_test), ndigits=2),  "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))

26/26 [==============================] - 0s 1ms/step
Alkalinity Training R2 = 0.91   MAE = 5.85   MSE = 71.52
Alkalinity Test     R2 = 0.85   MAE = 7.38   MSE = 96.28


In [20]:
root_logdir = os.path.join(os.curdir, "tf_training_logs")
tensorboard_cb = keras.callbacks.TensorBoard(root_logdir)

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=22))
ann.add(tf.keras.layers.Dense(units=44,  activation='sigmoid'))
ann.add(tf.keras.layers.Dense(units=88, activation= 'elu'))
ann.add(tf.keras.layers.Dense(units=44, activation= 'sigmoid'))
ann.add(tf.keras.layers.Dense(units=22, activation= 'relu'))
ann.add(tf.keras.layers.Dense(units=11, activation= 'relu'))
ann.add(tf.keras.layers.Dense(units=1))
ann.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.001), loss="mean_absolute_error", metrics=['mean_absolute_error'])
history = ann.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[keras.callbacks.EarlyStopping( monitor="val_loss", patience=100, mode="min", restore_best_weights=True),tensorboard_cb],batch_size=32, epochs=5000, verbose=0)


y_pred_test = ann.predict(X_test)
y_pred_train = ann.predict(X_train)

print(Ion, "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(mean_absolute_error(
    y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
print(Ion, "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2),  "  MAE =", round(mean_absolute_error(
    y_test, y_pred_test), ndigits=2),  "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))

26/26 [==============================] - 0s 1000us/step
Alkalinity Training R2 = 0.91   MAE = 5.68   MSE = 67.78
Alkalinity Test     R2 = 0.86   MAE = 7.23   MSE = 89.51


# Save Models

In [8]:
y_pred_ANN = ann.predict(X)
dfmerged['ANN'] = y_pred_ANN
dfmerged['LR'] = y_pred_LR
dfmerged['RT'] = y_pred_RT
dfmerged['RF'] = y_pred_RF
dfmerged['GB'] = y_pred_GB

33/33 [==============================] - 0s 1ms/step


In [9]:
Train, Test= train_test_split(dfmerged, test_size=0.2, random_state=1)
Train["Train|Test"]='Train'
Test["Train|Test"]='Test'
dfmerged2=pd.concat([Train,Test])
dfmerged2= dfmerged2.loc[:, ['Long Station Name','Collection Date','EC',Ion,'Sacramento X2','Location','WYT','Data','month','year', 'ANN','LR','RT','RF','GB','Train|Test']]
#dfmerged2.sort_values('Collection Date')
dfmerged2.to_csv(f'Output/{Ion}.csv')

In [297]:
# print('Train')
# print('MLR:', 'R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['LR']), ndigits=2), "  MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['LR']), ndigits=2), " MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['LR']), ndigits=2))
# print('RT:', ' R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RT']), ndigits=2), "  MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RT']), ndigits=2), "  MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RT']), ndigits=2))
# print('RF:', ' R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RF']), ndigits=2), "  MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RF']), ndigits=2), "  MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RF']), ndigits=2))
# print('GB:', ' R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['GB']), ndigits=2), "  MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['GB']), ndigits=2), "   MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['GB']), ndigits=2))
# print('ANN:', 'R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['ANN']), ndigits=2), "  MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['ANN']), ndigits=2), "  MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['ANN']), ndigits=2))

# print('Test')
# print('MLR:', 'R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['LR']), ndigits=2), "  MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['LR']), ndigits=2), " MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['LR']), ndigits=2))
# print('RT:', ' R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RT']), ndigits=2), "   MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RT']), ndigits=2), "  MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RT']), ndigits=2))
# print('RF:', ' R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RF']), ndigits=2), "   MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RF']), ndigits=2), "  MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RF']), ndigits=2))
# print('GB:', ' R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['GB']), ndigits=2), "  MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['GB']), ndigits=2), "  MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['GB']), ndigits=2))
# print('ANN:', 'R2:',round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['ANN']), ndigits=2), "   MAE =", round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['ANN']), ndigits=2), "  MSE =", round(mean_squared_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['ANN']), ndigits=2))



In [10]:
Performance= pd.DataFrame(index=["MLR", "RT", "GB","RF","ANN"],columns=['Train_R2','Train_MAE','Test_R2',"Test_MAE"])
Performance.loc["MLR"]["Train_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['LR']), ndigits=2)
Performance.loc["RT"]["Train_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RT']), ndigits=2)
Performance.loc["GB"]["Train_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['GB']), ndigits=2)
Performance.loc["RF"]["Train_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RF']), ndigits=2)
Performance.loc["ANN"]["Train_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['ANN']), ndigits=2)

Performance.loc["MLR"]["Train_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['LR']), ndigits=2)
Performance.loc["RT"]["Train_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RT']), ndigits=2)
Performance.loc["GB"]["Train_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['GB']), ndigits=2)
Performance.loc["RF"]["Train_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['RF']), ndigits=2)
Performance.loc["ANN"]["Train_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Train"][Ion], dfmerged2[dfmerged2['Train|Test']=="Train"]['ANN']), ndigits=2)

Performance.loc["MLR"]["Test_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['LR']), ndigits=2)
Performance.loc["RT"]["Test_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RT']), ndigits=2)
Performance.loc["GB"]["Test_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['GB']), ndigits=2)
Performance.loc["RF"]["Test_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RF']), ndigits=2)
Performance.loc["ANN"]["Test_MAE"]=round(mean_absolute_error(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['ANN']), ndigits=2)

Performance.loc["MLR"]["Test_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['LR']), ndigits=2)
Performance.loc["RT"]["Test_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RT']), ndigits=2)
Performance.loc["GB"]["Test_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['GB']), ndigits=2)
Performance.loc["RF"]["Test_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['RF']), ndigits=2)
Performance.loc["ANN"]["Test_R2"]=round(r2_score(dfmerged2[dfmerged2['Train|Test']=="Test"][Ion], dfmerged2[dfmerged2['Train|Test']=="Test"]['ANN']), ndigits=2)
Performance.to_csv("Performance.csv")
Performance

,Train_R2,Train_MAE,Test_R2,Test_MAE
MLR,0.76,10.74,0.65,10.84
RT,0.95,4.61,0.8,8.51
GB,0.93,5.4,0.86,7.33
RF,0.95,4.2,0.86,6.88
ANN,0.93,4.77,0.86,7.32


# Save Models

In [299]:
ann.save(f'Models/ANN_{Ion}.h5')
import joblib
joblib.dump(regressorLinear,f'Models/MLR_{Ion}.pkl')
joblib.dump(regressorTree,f'Models/RT_{Ion}.pkl')
joblib.dump(regressorRF,f'Models/RF_{Ion}.pkl')
joblib.dump(regressorGB,f'Models/GB_{Ion}.pkl')

joblib.dump(list(X.columns),f'Models/col_names.pkl')

['Models/col_names.pkl']

# K-fold cross validation

k=5 in this case (kinda same as the 0.2 setting above)

# RF

In [300]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, random_state=1, shuffle=True)

for train_index, test_index in kf.split(X):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    regressorRFk = RandomForestRegressor(n_estimators=100, random_state=0, min_samples_leaf=4)
    regressorRFk.fit(X_train, y_train)
    y_pred_test = regressorRFk.predict(X_test)
    y_pred_train = regressorRFk.predict(X_train)
    y_pred_RF = regressorRFk.predict(X)
    y_pred_test_RF = regressorRFk.predict(X_test)

    print("Random Forest:")
    print("        ", "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(
        mean_absolute_error(y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
    print("        ", "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2), "  MAE =", round(
        mean_absolute_error(y_test, y_pred_test), ndigits=2), "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))

TRAIN: 1068 TEST: 268
Random Forest:
         Training R2 = 0.96   MAE = 0.73   MSE = 3.09
         Test     R2 = 0.95   MAE = 0.95   MSE = 3.63
TRAIN: 1069 TEST: 267
Random Forest:
         Training R2 = 0.97   MAE = 0.65   MSE = 1.82
         Test     R2 = 0.89   MAE = 1.12   MSE = 9.44
TRAIN: 1069 TEST: 267
Random Forest:
         Training R2 = 0.96   MAE = 0.73   MSE = 2.92
         Test     R2 = 0.97   MAE = 0.89   MSE = 1.47
TRAIN: 1069 TEST: 267
Random Forest:
         Training R2 = 0.96   MAE = 0.72   MSE = 3.05
         Test     R2 = 0.94   MAE = 1.12   MSE = 4.6
TRAIN: 1069 TEST: 267
Random Forest:
         Training R2 = 0.96   MAE = 0.7   MSE = 2.94
         Test     R2 = 0.95   MAE = 0.99   MSE = 4.49


In [301]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, random_state=1, shuffle=True)
KfoldM = pd.DataFrame(columns=['R2_Train', 'MAE_Train','R2_Test', 'MAE_Test'])
KFoldR2Test=[]
KFoldMAETest=[]
KFoldR2Train=[]
KFoldMAETrain=[]
for train_index, test_index in kf.split(X):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    ann = tf.keras.models.Sequential()
    ann.add(tf.keras.layers.Dense(units=22))
    ann.add(tf.keras.layers.Dense(units=N[0], activation=A[0]))
    ann.add(tf.keras.layers.Dense(units=N[1], activation=A[1]))
    ann.add(tf.keras.layers.Dense(units=N[2], activation=A[2]))
    ann.add(tf.keras.layers.Dense(units=N[3], activation=A[3]))
    ann.add(tf.keras.layers.Dense(units=1))
    ann.compile(optimizer=keras.optimizers.Adamax(learning_rate=0.001), loss="mean_absolute_error", metrics=['mean_absolute_error'])
    history = ann.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[keras.callbacks.EarlyStopping( monitor="val_loss", patience=100, mode="min", restore_best_weights=True),tensorboard_cb],batch_size=32, epochs=5000, verbose=0)
    y_pred_test = ann.predict(X_test)
    y_pred_train = ann.predict(X_train)
    y_pred_ANN = ann.predict(X)
    y_pred_test_ANN = ann.predict(X_test)
    KFoldR2Test.append(round(r2_score(y_test, y_pred_test), ndigits=2))
    KFoldMAETest.append(round(mean_absolute_error(y_test, y_pred_test), ndigits=2))
    KFoldR2Train.append(round(r2_score(y_train, y_pred_train), ndigits=2))
    KFoldMAETrain.append(round(mean_absolute_error(y_train, y_pred_train), ndigits=2))
    print("ANN:")
    print("        ", "Training", "R2 =", round(r2_score(y_train, y_pred_train), ndigits=2), "  MAE =", round(
        mean_absolute_error(y_train, y_pred_train), ndigits=2), "  MSE =", round(mean_squared_error(y_train, y_pred_train), ndigits=2))
    print("        ", "Test    ", "R2 =", round(r2_score(y_test, y_pred_test), ndigits=2), "  MAE =", round(
        mean_absolute_error(y_test, y_pred_test), ndigits=2), "  MSE =", round(mean_squared_error(y_test, y_pred_test), ndigits=2))

KFoldR2Test=pd.DataFrame(KFoldR2Test)
KFoldMAETest=pd.DataFrame(KFoldMAETest)
KFoldR2Train=pd.DataFrame(KFoldR2Train)
KFoldMAETrain=pd.DataFrame(KFoldMAETrain)
KfoldM["R2_Train"]=KFoldR2Train
KfoldM["MAE_Train"]=KFoldMAETrain
KfoldM["R2_Test"]=KFoldR2Test
KfoldM["MAE_Test"]=KFoldMAETest
KfoldM.to_csv("Kfold.csv")
KfoldM

TRAIN: 1068 TEST: 268
9/9 [==============================] - 0s 875us/step
ANN:
         Training R2 = 0.97   MAE = 0.67   MSE = 1.94
         Test     R2 = 0.97   MAE = 0.83   MSE = 2.06
TRAIN: 1069 TEST: 267
9/9 [==============================] - 0s 1ms/step
ANN:
         Training R2 = 0.99   MAE = 0.56   MSE = 1.02
         Test     R2 = 0.95   MAE = 0.95   MSE = 4.57
TRAIN: 1069 TEST: 267
9/9 [==============================] - 0s 1ms/step
ANN:
         Training R2 = 0.96   MAE = 0.75   MSE = 3.51
         Test     R2 = 0.98   MAE = 0.79   MSE = 1.1
TRAIN: 1069 TEST: 267
9/9 [==============================] - 0s 875us/step
ANN:
         Training R2 = 0.98   MAE = 0.64   MSE = 1.74
         Test     R2 = 0.98   MAE = 0.9   MSE = 1.67
TRAIN: 1069 TEST: 267
